In [6]:
BASE_URL = "https://ae.kickscrew.com/collections/nike"
CLASS_HITS_ITEM = "ais-Hits-item"
CLASS_PAGINATION_LINK = "ais-Pagination-item--lastPage"
csv_file_path = "products_urls.csv"

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv

data_list = []

try:
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    driver = webdriver.Chrome(options=options)


    wait = WebDriverWait(driver, 5)  # Adjust the timeout as needed
    driver.get(BASE_URL)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, CLASS_PAGINATION_LINK)))


    # Find the last page number
    last_page_element = driver.find_element(By.CLASS_NAME, CLASS_PAGINATION_LINK)
    last_page_url = last_page_element.find_element(By.TAG_NAME,"a").get_attribute("href")
    last_page = int(last_page_url.split("page=")[-1])

    # Generate the list of URLs
    urls = [BASE_URL + "?page=" + str(i) for i in range(310, last_page + 1)]

    product_urls = []
    
    for url in urls:
        driver.get(url)

        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, CLASS_HITS_ITEM)))

        search_results = driver.find_elements(By.CLASS_NAME,CLASS_HITS_ITEM)

        product_urls.extend([result.find_element(By.TAG_NAME,"a").get_attribute("href") for result in search_results])

    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Product URL']
        csv_writer = csv.writer(csv_file)

        csv_writer.writerow(fieldnames)  # Write the header

        # Write each product URL as a separate row
        for url in product_urls:
            csv_writer.writerow([url])

    print(f"Data has been saved to {csv_file_path}")

finally:
    driver.quit()


Data has been saved to products_urls.csv


In [2]:
import csv
urls_file_path = "products_urls.csv"

products_urls = []

with open(urls_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    # Skip the header row if it exists
    next(csv_reader, None)

    # Read each row and append the product URL to the list
    for row in csv_reader:
        products_urls.append(row[0])  # Assuming the URL is in the first (0-indexed) column

products_urls = [url.replace(".com",'.com/en') for url in products_urls]

In [5]:
file_path = "scraped_data.json"

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
from bs4 import BeautifulSoup
import re

def find_product_info(soup):
    product_info_raw = soup.find('div', class_="pdp-product-info-container").get_text().strip().split('\n\n')
    product_info = {}
    for item in product_info_raw:
        key, value = item.split('\n', 1)  # Split only at the first occurrence of newline
        product_info[key] = value
    return product_info

def find_gender(soup):
    pdp_tab_header = soup.find('div', class_='pdpTabHeader')
    gender = pdp_tab_header.find('div', class_='pdpOptionValues').text.strip()
    return gender

data = []

try:
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed

    for url in products_urls[:6]:
        driver.get(url)
        
        page_source = driver.page_source
        
        soup = BeautifulSoup(page_source, 'html.parser')
        product_data = {}
        product_data['handle'] = url.split('/products/')[-1]
        product_data['title'] = soup.find('h1', class_='product-area__details__title').get_text(strip=True)
        element = soup.find('div', class_="cc-tabs__tab__panel",id = re.compile("product-tab-panel3_"))
        if element:
            product_data['body'] = element.get_text(strip=True)
        else:
            print("Body not found.")        
        product_data['type'] = soup.find('div', class_="vendor product-detail__gap-sm").get_text(strip=True)
        product_data['product_info'] = find_product_info(soup)
        product_data['gender'] = find_gender(soup)
        data.append(product_data)
    with open(file_path,'w') as file:
        json.dump(data, file, indent=2)
    print(f'Data has been written to {file_path}')
    

finally:
    driver.quit()

Data has been written to scraped_data.json
